# Harnessing Weather Insights for Accurate Energy Load Forecasting

## Download Training Data

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [44]:
# API Settings

# Transparency Portal
TRANSPARENCY_API_KEY = "9d0ebad5-08cf-4d6f-a752-744ba3707b70"
TRANSPARENCY_PERIOD_START = "2024-01-01"
TRANSPARENCY_PERIOD_END = "2024-12-31"
TRANSPARENCY_BIDDING_ZONE = "10YAT-APG------L" # Load Austria

# GeoSphere
GEOSPHERE_PERIOD_START = "2024-01-01"
GEOSPHERE_PERIOD_END = "2024-12-31"
GEOSPHERE_STATIONS = "1, 105"

In [46]:
# Transparency Portal
import requests
import datetime
import os

if TRANSPARENCY_API_KEY == "DEMO_KEY":
    print("Error: Missing Transparency API key.")
else:
    api_url = "https://web-api.tp.entsoe.eu/api"
    
    start = datetime.datetime.strptime(TRANSPARENCY_PERIOD_START, "%Y-%m-%d")
    end = datetime.datetime.strptime(TRANSPARENCY_PERIOD_END, "%Y-%m-%d")
    
    # Go through each month of the span
    current = start
    while current <= end:
        print("Requesting data for " + current.strftime("%Y-%m"))
        
        next_month = (current.replace(day=1) + datetime.timedelta(days=32)).replace(day=1)
        period_end = min(next_month, end + datetime.timedelta(days=1))
        
        api_params = {
            "documentType": "A65", # System Total Load
            "processType": "A16", # Actual Realized Load
            "outBiddingZone_Domain": TRANSPARENCY_BIDDING_ZONE,
            "periodStart": current.strftime("%Y%m%d0000"), # Format: YYYYMMDDHHMM
            "periodEnd": period_end.strftime("%Y%m%d0000"), # Format: YYYYMMDDHHMM
            "securityToken": TRANSPARENCY_API_KEY
        }
        
        response = requests.get(api_url, params=api_params) # Send the request => Get XML response

        if response.status_code == 200:
            # Save to file and create directory if it doesn't exist
            year_folder = current.strftime("%Y")
            month_file = current.strftime("%m")
            os.makedirs(os.path.dirname(f"./data/transparency/{year_folder}/"), exist_ok=True)
            
            with open(f"./data/transparency/{year_folder}/{month_file}.xml", "w", encoding='utf-8') as file:
                file.write(response.text)
                
            print(f"Data saved to ./data/transparency/{year_folder}/{month_file}.xml")
                
        else:
            print("Error: " + str(response.status_code))
            print("Response: " + response.text)
        
        current = next_month


Requesting data for 2024-01
Data saved to ./data/transparency/2024/01.xml
Requesting data for 2024-02
Data saved to ./data/transparency/2024/02.xml
Requesting data for 2024-03
Data saved to ./data/transparency/2024/03.xml
Requesting data for 2024-04
Data saved to ./data/transparency/2024/04.xml
Requesting data for 2024-05
Data saved to ./data/transparency/2024/05.xml
Requesting data for 2024-06
Data saved to ./data/transparency/2024/06.xml
Requesting data for 2024-07
Data saved to ./data/transparency/2024/07.xml
Requesting data for 2024-08
Data saved to ./data/transparency/2024/08.xml
Requesting data for 2024-09
Data saved to ./data/transparency/2024/09.xml
Requesting data for 2024-10
Data saved to ./data/transparency/2024/10.xml
Requesting data for 2024-11
Data saved to ./data/transparency/2024/11.xml
Requesting data for 2024-12
Data saved to ./data/transparency/2024/12.xml


In [31]:
# GeoSphere Historical Data
api_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/"
api_dataset = "klima-v2-1d"
api_params = {
    # Data to get
    "parameters": "rr,tl_mittel,bewm_mittel,so_h,vv_mittel", # Rainfall, Temperature, Cloudiness, Sunshine, Wind speed
    
    "start": GEOSPHERE_PERIOD_START,
    "end": GEOSPHERE_PERIOD_END,
    "station_ids": GEOSPHERE_STATIONS,
    "output_format": "csv",
}
# Download the data to a file
response = requests.get(api_url + api_dataset, params=api_params) # Send the request => Get GeoJSON response

if response.status_code == 200:
    with open("./data/geosphere.csv", "wb") as file:
        file.write(response.content)
    print("Data saved to file")
else:
    print("Error: " + str(response.status_code))
    print("Response: " + response.text)

Data saved to file
